In [2]:
import pandas as pd

In [3]:
# https://data.ace.uq.edu.au/public/gtdb/data/releases/release220/220.0/bac120_taxonomy_r220.tsv

In [10]:
bracken_species=pd.read_csv(
    "/shared/projects/mudis4ls_is4_benchmark/test_results/kraken/gtdb/test_Species.bracken", sep="\t"
    )

In [11]:
bracken_species

,name,taxonomy_id,taxonomy_lvl,kraken_assigned_reads,added_reads,new_est_reads,fraction_total_reads
0,Bacteroides stercoris,46506,S,3744,2,3746,0.07276
1,Bacteroides uniformis,820,S,2991,0,2991,0.05809
2,Bacteroides xylanisolvens,371601,S,1448,0,1448,0.02812
3,Bacteroides finegoldii,338188,S,994,0,994,0.01931
4,Bacteroides ovatus,28116,S,794,0,794,0.01542
...,...,...,...,...,...,...,...
1077,Methanolobus mangrovi,3072977,S,1,0,1,0.00002
1078,Homo sapiens,9606,S,5,0,5,0.00010
1079,Junduvirus communis,2956034,S,10,0,10,0.00019
1080,Junduvirus copri,2956035,S,1,0,1,0.00002


In [12]:


# Charger les fichiers GTDB pour les bactéries et les archées
bac120_gtdb = pd.read_csv(
    "/shared/projects/mudis4ls_is4_benchmark/gtdb/bac120_taxonomy_r220.tsv",
    sep="\t", header=None, names=["species_id", "taxonomy"]
)

ar53_gtdb = pd.read_csv(
    "/shared/projects/mudis4ls_is4_benchmark/gtdb/ar53_taxonomy_r220.tsv",
    sep="\t", header=None, names=["species_id", "taxonomy"]
)

# Extraire les genres (g__) pour bactéries et archées
bac120_gtdb["genus"] = bac120_gtdb["taxonomy"].str.extract(r";g__([^;]+)")
ar53_gtdb["genus"] = ar53_gtdb["taxonomy"].str.extract(r";g__([^;]+)")

# Fusionner les genres des deux fichiers en un seul ensemble
known_genera = set(pd.concat([bac120_gtdb["genus"], ar53_gtdb["genus"]]).dropna().unique())

# Vérification : Afficher les 10 premiers genres valides
print("Exemple de genres valides:", list(known_genera)[:10])




Exemple de genres valides: ['4484-190-2', 'MarineAlpha5-Bin12', 'JABDMV01', 'Millisia', 'Parazoarcus', 'RYN-404', 'SCTL01', 'JAMCWI01', 'AG-363-K07', 'DMER64']


In [13]:
### Fonction mise à jour pour extraire le genre et l'espèce ###
def extract_genus_species(species_name):
    words = species_name.split()

    # Si le premier mot est un genre connu, on l'extrait normalement
    if words[0] in known_genera:
        genus = words[0]
        species = " ".join(words[1:]) if len(words) > 1 else "sp."
    else:
        # Si inconnu : 
        # - genus = tout sauf le dernier mot
        # - species = dernier mot
        genus = " ".join(words[:-1]) if len(words) > 1 else words[0]
        species = words[-1]

    return genus, species

# Appliquer la fonction sur la colonne "species" du dataframe Bracken
bracken_species[["genus", "species"]] = bracken_species["name"].apply(
    lambda x: pd.Series(extract_genus_species(x))
)


In [14]:
bracken_species

,name,taxonomy_id,taxonomy_lvl,kraken_assigned_reads,added_reads,new_est_reads,fraction_total_reads,genus,species
0,Bacteroides stercoris,46506,S,3744,2,3746,0.07276,Bacteroides,stercoris
1,Bacteroides uniformis,820,S,2991,0,2991,0.05809,Bacteroides,uniformis
2,Bacteroides xylanisolvens,371601,S,1448,0,1448,0.02812,Bacteroides,xylanisolvens
3,Bacteroides finegoldii,338188,S,994,0,994,0.01931,Bacteroides,finegoldii
4,Bacteroides ovatus,28116,S,794,0,794,0.01542,Bacteroides,ovatus
...,...,...,...,...,...,...,...,...,...
1077,Methanolobus mangrovi,3072977,S,1,0,1,0.00002,Methanolobus,mangrovi
1078,Homo sapiens,9606,S,5,0,5,0.00010,Homo,sapiens
1079,Junduvirus communis,2956034,S,10,0,10,0.00019,Junduvirus,communis
1080,Junduvirus copri,2956035,S,1,0,1,0.00002,Junduvirus,copri


In [15]:
bracken_species.rename(columns={"fraction_total_reads": "kraken_fraction_total_reads"}, inplace=True)

In [16]:
bracken_essentials=bracken_species.drop(
    columns=["name", "taxonomy_id", "taxonomy_lvl","kraken_assigned_reads","added_reads","new_est_reads"]
    )

In [17]:
bracken_essentials

,kraken_fraction_total_reads,genus,species
0,0.07276,Bacteroides,stercoris
1,0.05809,Bacteroides,uniformis
2,0.02812,Bacteroides,xylanisolvens
3,0.01931,Bacteroides,finegoldii
4,0.01542,Bacteroides,ovatus
...,...,...,...
1077,0.00002,Methanolobus,mangrovi
1078,0.00010,Homo,sapiens
1079,0.00019,Junduvirus,communis
1080,0.00002,Junduvirus,copri


In [18]:
sylph_meteor=pd.read_csv(
    "/shared/projects/mudis4ls_is4_benchmark/test_results/compare_outputs/sylph_meteor_bracken/meteor_sylph.tsv", sep="\t"
    )

In [19]:
sylph_meteor

,msp_name,abundance_meteor,gtdb_r220_classification,superkingdom,phylum,class,order,family,genus,species,msp,abundance_sylph,sequence_abundance,ANI (if strain-level),Coverage (if strain-level)
0,NaN,NaN,d__Bacteria,Bacteria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0000,100.0001,NaN,NaN
1,msp_0584,0.006384,d__Bacteria;p__Actinomycetota;c__Coriobacterii...,Bacteria,Actinomycetota,Coriobacteriia,Coriobacteriales,Eggerthellaceae,Anaerotardibacter,Anaerotardibacter sp000435675,msp_0584,NaN,NaN,NaN,NaN
2,msp_0573,0.006947,d__Bacteria;p__Actinomycetota;c__Coriobacterii...,Bacteria,Actinomycetota,Coriobacteriia,Coriobacteriales,Eggerthellaceae,Eggerthella,Eggerthella lenta,msp_0573,NaN,NaN,NaN,NaN
3,NaN,NaN,d__Bacteria;p__Bacillota_A,Bacteria,Bacillota_A,NaN,NaN,NaN,NaN,NaN,NaN,41.4963,36.1829,NaN,NaN
4,NaN,NaN,d__Bacteria;p__Bacillota_A;c__Clostridia,Bacteria,Bacillota_A,Clostridia,NaN,NaN,NaN,NaN,NaN,41.4963,36.1829,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,NaN,NaN,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,NaN,NaN,NaN,1.3124,0.9629,NaN,NaN
135,NaN,NaN,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,Sutterella,NaN,NaN,1.3124,0.9629,NaN,NaN
136,msp_0280,0.030240,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,Sutterella,Sutterella wadsworthensis_A,msp_0280,1.3124,0.9629,NaN,NaN
137,NaN,NaN,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,Sutterella,Sutterella wadsworthensis_A,NaN,1.3124,0.9629,98.48,0.035


In [20]:
bracken_essentials["species"] = bracken_essentials["genus"] + " " + bracken_essentials["species"]


In [21]:
bracken_essentials

,kraken_fraction_total_reads,genus,species
0,0.07276,Bacteroides,Bacteroides stercoris
1,0.05809,Bacteroides,Bacteroides uniformis
2,0.02812,Bacteroides,Bacteroides xylanisolvens
3,0.01931,Bacteroides,Bacteroides finegoldii
4,0.01542,Bacteroides,Bacteroides ovatus
...,...,...,...
1077,0.00002,Methanolobus,Methanolobus mangrovi
1078,0.00010,Homo,Homo sapiens
1079,0.00019,Junduvirus,Junduvirus communis
1080,0.00002,Junduvirus,Junduvirus copri


In [22]:
sylph_meteor

,msp_name,abundance_meteor,gtdb_r220_classification,superkingdom,phylum,class,order,family,genus,species,msp,abundance_sylph,sequence_abundance,ANI (if strain-level),Coverage (if strain-level)
0,NaN,NaN,d__Bacteria,Bacteria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0000,100.0001,NaN,NaN
1,msp_0584,0.006384,d__Bacteria;p__Actinomycetota;c__Coriobacterii...,Bacteria,Actinomycetota,Coriobacteriia,Coriobacteriales,Eggerthellaceae,Anaerotardibacter,Anaerotardibacter sp000435675,msp_0584,NaN,NaN,NaN,NaN
2,msp_0573,0.006947,d__Bacteria;p__Actinomycetota;c__Coriobacterii...,Bacteria,Actinomycetota,Coriobacteriia,Coriobacteriales,Eggerthellaceae,Eggerthella,Eggerthella lenta,msp_0573,NaN,NaN,NaN,NaN
3,NaN,NaN,d__Bacteria;p__Bacillota_A,Bacteria,Bacillota_A,NaN,NaN,NaN,NaN,NaN,NaN,41.4963,36.1829,NaN,NaN
4,NaN,NaN,d__Bacteria;p__Bacillota_A;c__Clostridia,Bacteria,Bacillota_A,Clostridia,NaN,NaN,NaN,NaN,NaN,41.4963,36.1829,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,NaN,NaN,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,NaN,NaN,NaN,1.3124,0.9629,NaN,NaN
135,NaN,NaN,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,Sutterella,NaN,NaN,1.3124,0.9629,NaN,NaN
136,msp_0280,0.030240,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,Sutterella,Sutterella wadsworthensis_A,msp_0280,1.3124,0.9629,NaN,NaN
137,NaN,NaN,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,Sutterella,Sutterella wadsworthensis_A,NaN,1.3124,0.9629,98.48,0.035


In [23]:
sylph_meteor_bracken=pd.merge(
    sylph_meteor, bracken_essentials, on=("species","genus"), how="outer"
    )

In [24]:
sylph_meteor_bracken

,msp_name,abundance_meteor,gtdb_r220_classification,superkingdom,phylum,class,order,family,genus,species,msp,abundance_sylph,sequence_abundance,ANI (if strain-level),Coverage (if strain-level),kraken_fraction_total_reads
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abyssibius,Abyssibius alkaniclasticus,NaN,NaN,NaN,NaN,NaN,0.00002
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acetivibrio,Acetivibrio saccincola,NaN,NaN,NaN,NaN,NaN,0.00002
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acetivibrio,Acetivibrio thermocellus,NaN,NaN,NaN,NaN,NaN,0.00002
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acetobacter,Acetobacter aceti,NaN,NaN,NaN,NaN,NaN,0.00002
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acetobacter,Acetobacter persici,NaN,NaN,NaN,NaN,NaN,0.00002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183,NaN,NaN,d__Bacteria;p__Pseudomonadota;c__Alphaproteoba...,Bacteria,Pseudomonadota,Alphaproteobacteria,RF32,NaN,NaN,NaN,NaN,2.9883,1.3483,NaN,NaN,NaN
1184,NaN,NaN,d__Bacteria;p__Pseudomonadota;c__Alphaproteoba...,Bacteria,Pseudomonadota,Alphaproteobacteria,RF32,CAG-239,NaN,NaN,NaN,2.9883,1.3483,NaN,NaN,NaN
1185,NaN,NaN,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,Bacteria,Pseudomonadota,Gammaproteobacteria,NaN,NaN,NaN,NaN,NaN,1.3124,0.9629,NaN,NaN,NaN
1186,NaN,NaN,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,NaN,NaN,NaN,NaN,1.3124,0.9629,NaN,NaN,NaN


In [29]:
colonnes_a_garder = [
    "abundance_meteor", "gtdb_r220_classification", "species","abundance_sylph","kraken_fraction_total_reads"
    ]


In [30]:
sylph_meteor_bracken_species = sylph_meteor_bracken[colonnes_a_garder]


In [32]:
sylph_meteor_bracken_species.to_csv(
    "/shared/projects/mudis4ls_is4_benchmark/test_results/compare_outputs/sylph_meteor_bracken/sylph_meteor_bracken_species.tsv", sep="\t", index=False
    )

sylph_meteor_bracken_species

In [33]:
sylph_meteor_bracken_species

,abundance_meteor,gtdb_r220_classification,species,abundance_sylph,kraken_fraction_total_reads
0,NaN,NaN,Abyssibius alkaniclasticus,NaN,0.00002
1,NaN,NaN,Acetivibrio saccincola,NaN,0.00002
2,NaN,NaN,Acetivibrio thermocellus,NaN,0.00002
3,NaN,NaN,Acetobacter aceti,NaN,0.00002
4,NaN,NaN,Acetobacter persici,NaN,0.00002
...,...,...,...,...,...
3174,NaN,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,NaN,1.3124,0.00004
3175,NaN,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,NaN,1.3124,0.00002
3176,NaN,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,NaN,1.3124,0.00010
3177,NaN,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,NaN,1.3124,0.00019
